In [30]:
# ARC Simulation
from __future__ import division

from platform import python_version
print(python_version())

import argparse
import meep as mp
import time
import numpy as np
import matplotlib.pyplot as plt
from meep.materials import  cSi, SiO2
#from mayavi import mlab


3.9.6


In [31]:
sx=10
sy=10
sz=10
eps_Si=12
eps_air=1
eps=4
T_Si=2
T_Arc=0.1
dpml = 1.0
cell = mp.Vector3(sx, sy, sz)
pml_layers = [mp.PML(dpml)]
resolution = 15
nfreq = 200
fcen = 1.75  # pulse center frequency
df = 1.5  # pulse width (in frequency)

In [32]:
#geometry = [mp.Block(mp.Vector3(T_Si, sy, mp.inf), center=mp.Vector3(0, 0), material=mp.Medium(epsilon=eps_Si))]
geometry = [mp.Block(
                     size = mp.Vector3(sx, sy, sz), 
                     center = mp.Vector3(0, 0, 0), 
                     material=mp.Medium(epsilon=eps_air)
                    )
           ]

In [33]:
sources = [mp.Source(
                     mp.GaussianSource(fcen, fwidth=df), 
                     component=mp.Ez, 
                     center=mp.Vector3(1 + (-0.5 * sx), 0, 0), 
                     size=mp.Vector3(0, sy, sz)
                    )
          ]

In [34]:
sim = mp.Simulation(cell_size=cell,
                    boundary_layers=pml_layers,
                    geometry=geometry,
                    sources=sources,
                    resolution=resolution)

refl_1_fr = mp.FluxRegion(center=mp.Vector3(0.5*sx-dpml-T_Si-1.1*T_Arc, 0, 0), 
                          size=mp.Vector3(0, sy, sz)
                         )
refl_1 = sim.add_flux(fcen, df, nfreq, refl_1_fr)

tran_1_fr = mp.FluxRegion(center=mp.Vector3(0.5*sx-dpml, 0, 0), 
                          size=mp.Vector3(0, sy, sz)
                         )
tran_1 = sim.add_flux(fcen, df, nfreq, tran_1_fr)
#plt.figure(dpi=600)
#sim.plot3D()
#plt.show()

In [35]:
pt = mp.Vector3(0.5*sx-dpml-0.1, 0, 0)
sim.run(mp.at_beginning(mp.output_epsilon),
                until_after_sources=mp.stop_when_fields_decayed(50, mp.Ez, pt, 1e-3))
no_ARC_refl_data = sim.get_flux_data(refl_1)

-----------
Initializing structure...
time for choose_chunkdivision = 0.000381947 s
Working in 3D dimensions.
Computational cell is 10 x 10 x 10 with resolution 15
     block, center = (0,0,0)
          size (10,10,10)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1,1,1)
time for set_epsilon = 9.7102 s
-----------
creating output file "./eps-000000.00.h5"...
on time step 5 (time=0.166667), 0.823475 s/step
on time step 30 (time=1), 0.164247 s/step
on time step 55 (time=1.83333), 0.164005 s/step
on time step 80 (time=2.66667), 0.163108 s/step
on time step 105 (time=3.5), 0.163105 s/step
on time step 130 (time=4.33333), 0.163476 s/step
on time step 155 (time=5.16667), 0.164697 s/step
on time step 180 (time=6), 0.163502 s/step
on time step 205 (time=6.83333), 0.163143 s/step
on time step 230 (time=7.66667), 0.163419 s/step
on time step 254 (time=8.46667), 0.168181 s/step
on time step 278 (time=9.26667), 0.167045 s/step
on time step 302 (time=10.

In [36]:
no_ARC_tran_flux = mp.get_fluxes(tran_1)
no_ARC_refl_flux = mp.get_fluxes(refl_1)

In [37]:
sim.reset_meep()
geometry = [mp.Block(
                    size=mp.Vector3(T_Si, sy, sz), 
                    center=mp.Vector3(0.5*sx-dpml-0.5*T_Si, 0, 0),
                    material=cSi
                    )
           ]
sim = mp.Simulation(
                    cell_size=cell,
                    boundary_layers=pml_layers,
                    geometry=geometry,
                    sources=sources,
                    resolution=resolution
                   )

#refl_1_fr = mp.FluxRegion(
#                          center=mp.Vector3(0.5*sx-dpml-0.5*T_Si-1.1*T_Arc, 0, 0),
#                          size=mp.Vector3(0, sy, sz)
#                          )
refl_1 = sim.add_flux(fcen, df, nfreq, refl_1_fr)

tran_1_fr = mp.FluxRegion(
                         center=mp.Vector3(0.5*sx-dpml, 0, 0), 
                         size=mp.Vector3(0, sy, sz)
                         )
tran_1 = sim.add_flux(fcen, df, nfreq, tran_1_fr)

sim.load_minus_flux_data(refl_1, no_ARC_refl_data)
                          
pt = mp.Vector3(0.5*sx-dpml-0.1, 0, 0)
sim.run(mp.at_beginning(mp.output_epsilon),
                until_after_sources=mp.stop_when_fields_decayed(50, mp.Ez, pt, 1e-3))

with_ARC_refl_flux = mp.get_fluxes(refl_1)
with_ARC_tran_flux = mp.get_fluxes(tran_1)

flux_freqs = mp.get_flux_freqs(refl_1)

#plt.figure(dpi=600)
#sim.plot2D()
#plt.show()


-----------
Initializing structure...
time for choose_chunkdivision = 0.000349045 s
Working in 3D dimensions.
Computational cell is 10 x 10 x 10 with resolution 15
     block, center = (3,0,0)
          size (2,10,10)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1,1,1)
time for set_epsilon = 6.14924 s
lorentzian susceptibility: frequency=1.73, gamma=5
lorentzian susceptibility: frequency=2.76, gamma=0.126
lorentzian susceptibility: frequency=3.64, gamma=0
-----------
creating output file "./eps-000000.00.h5"...
on time step 1 (time=0.0333333), 4.91357 s/step
on time step 16 (time=0.533333), 0.26754 s/step
on time step 32 (time=1.06667), 0.264216 s/step
on time step 48 (time=1.6), 0.263174 s/step
on time step 64 (time=2.13333), 0.263765 s/step
on time step 80 (time=2.66667), 0.264495 s/step
on time step 96 (time=3.2), 0.263875 s/step
on time step 111 (time=3.7), 0.26841 s/step
on time step 127 (time=4.23333), 0.266579 s/step
on time step 142

RuntimeError: meep: simulation fields are NaN or Inf

In [18]:
no_ARC_refl_data

FluxData(E=array([-5.92206527e-07-1.36925186e-06j, -4.62214168e-07-1.45152235e-06j,
       -3.21789660e-07-1.52191811e-06j, ...,
        2.61925601e-11-1.30876629e-12j,  2.56388167e-11+1.53474454e-12j,
        2.47937894e-11+4.24369214e-12j]), H=array([-2.15453995e-04-4.94675996e-04j, -1.68173037e-04-5.24807080e-04j,
       -1.17038031e-04-5.50576177e-04j, ...,
        8.04118247e-19+9.98154838e-19j, -3.15758982e-19+1.40784313e-18j,
       -1.32140393e-18+7.42708819e-19j]))

In [19]:
no_ARC_tran_flux

[4.341496395219716e-05,
 4.5298752826755754e-05,
 4.726043404026932e-05,
 4.930307070087313e-05,
 5.142984239833831e-05,
 5.364403843093555e-05,
 5.5949054809997804e-05,
 5.834839982514192e-05,
 6.0845707468383916e-05,
 6.344475278511076e-05,
 6.614946176809303e-05,
 6.896391186754111e-05,
 7.189232542644749e-05,
 7.49390632858327e-05,
 7.810862606835706e-05,
 8.140566587953857e-05,
 8.483500436829461e-05,
 8.840164875119312e-05,
 9.21107985417321e-05,
 9.596784178565953e-05,
 9.997834671217062e-05,
 0.00010414805812678363,
 0.0001084829051607986,
 0.00011298901973142792,
 0.0001176727579602553,
 0.0001225407146246748,
 0.0001275997251297597,
 0.00013285685777779973,
 0.00013831940585986622,
 0.0001439948897069884,
 0.00014989107266918258,
 0.0001560159859211774,
 0.0001623779510056506,
 0.00016898559025658446,
 0.0001758478231471368,
 0.0001829738561252821,
 0.00019037317830672225,
 0.000198055572079703,
 0.00020603113816099503,
 0.00021431032507819077,
 0.00022290394986245753,
 0.000

In [28]:
with_ARC_refl_flux

NameError: name 'with_ARC_refl_flux' is not defined

In [6]:
Wl = []
Rs = []
Ts = []
flux_freqs = mp.get_flux_freqs(refl_1)
for i in range(nfreq):
    wl = np.append(wl, 1/flux_freqs[i])
    Rs = np.append(Rs,-with_ARC_refl_flux[i]/no_ARC_refl_flux[i])
    Ts = np.append(Ts,with_ARC_tran_flux[i]/no_ARC_refl_flux[i]) 

NameError: name 'refl_1' is not defined

In [2]:
for i in range(nfreq):
    wl = np.append(wl, 1/flux_freqs[i])
    Rs = np.append(Rs,-with_ARC_refl_flux[i]/no_ARC_refl_flux[i])
    Ts = np.append(Ts,with_ARC_tran_flux[i]/no_ARC_refl_flux[i]) 

NameError: name 'nfreq' is not defined

In [7]:
no_ARC_refl_flux

NameError: name 'no_ARC_refl_flux' is not defined